# Aqui ficarão as anotações de controle preditivo

As anotações sobre o controlador preditivo aqui implementado está em anotações no caderno que posterirmente serão passadas para cá.

Será feito o uso do drake visualizer e do pacote plots para exibição dos resultados.